## Решение задачи кредитного скоринга с использованием полностью гомоморфного шифрования

В данной работе рассматривается ситуация, когда в банке возникла нехватка вычислительной мощности для решения задач машинного обучения, поэтому было принято решение арендовать для этих целей облачный сервер. 

Однако так как данные передаются на сторонний сервер, то возникает проблема безопасности данных, так как уязвимости в самом облаке, возможности несанкционированного доступа или даже неправильная настройка доступа к облаку могут привести к угрозам информационной безопасности, например, утечке данных, что снизит уровень доверия клиентов, а также приведёт к репутационным потерям и штрафам за нарушение правил информационной безопасности.

Решить данную проблему можно с помощью машинного обучения на зашифрованных данных, которое возможно с использованием полностью гомоморфного шифрования, позволяющего выполнять вычисления над зашифрованными данными без их расшифровки, что обеспечивает наивысший уровень безопасности, так как содержание данных никогда не раскрывается.

Демонстрация применения полностью гомоморфного шифрования для машинного обучения предлагается на примере решения задачи кредитного скоринга с предварительным шифрованием данных на стороне клиента (этап 1) и обучением и оценкой модели на зашифрованных данных на стороне сервера (этап 2). 

В качестве библиотеки полностью гомоморфного шифрования используется выбрана TenSEAL, основанная на Microsoft SEAL, которая в настоящее время является наиболее используемой библиотекой полностью гомоморфного шифрования. С помощью API она обеспечивает простоту использования языка Python, при этом сохраняя эффективность за счет реализации большинства операций с использованием C++.

### Этап 1 - Подготовка данных (клиент)

#### 1 Подготовка
##### Импорт библиотек

In [240]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tenseal as ts
import time
import torch

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.calibration import LabelEncoder
from sklearn.impute import SimpleImputer

##### Чтение данных
Набор данных [Credit score classification](https://www.kaggle.com/datasets/parisrohan/credit-score-classification/data), скачанный с Kaggle, выбран потому, что он включает в себя практически полную информацию о банковских клиентах, в том числе, возраст, профессию, годовой и ежемесячный доход, количество банковских счетов и так далее.

In [241]:
df = pd.read_csv('Credit score data.csv', low_memory=False)
df.tail()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
99995,0x25fe9,CUS_0x942c,April,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,_,502.38,34.663572,31 Years and 6 Months,No,35.104023,60.97133255718485,High_spent_Large_value_payments,479.86622816574095,Poor
99996,0x25fea,CUS_0x942c,May,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,_,502.38,40.565631,31 Years and 7 Months,No,35.104023,54.18595028760385,High_spent_Medium_value_payments,496.651610435322,Poor
99997,0x25feb,CUS_0x942c,June,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Good,502.38,41.255522,31 Years and 8 Months,No,35.104023,24.02847744864441,High_spent_Large_value_payments,516.8090832742814,Poor
99998,0x25fec,CUS_0x942c,July,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Good,502.38,33.638208,31 Years and 9 Months,No,35.104023,251.67258219721603,Low_spent_Large_value_payments,319.1649785257098,Standard
99999,0x25fed,CUS_0x942c,August,Nicks,25,078-73-5990,Mechanic,39628.99_,3359.415833,4,...,Good,502.38,34.192463,31 Years and 10 Months,No,35.104023,167.1638651610451,!@9#%8,393.6736955618808,Poor


#### 2 Очистка данных

In [242]:
print('Размер набора данных: ', df.shape)

Размер набора данных:  (100000, 28)


In [243]:
df.describe(exclude=np.number).T

,count,unique,top,freq
ID,100000,100000,0x1602,1
Customer_ID,100000,12500,CUS_0xd40,8
Month,100000,8,January,12500
Name,90015,10139,Langep,44
Age,100000,1788,38,2833
SSN,100000,12501,#F%$D@*&8,5572
Occupation,100000,16,_______,7062
Annual_Income,100000,18940,36585.12,16
Num_of_Loan,100000,434,3,14386
Type_of_Loan,88592,6260,Not Specified,1408


Выводы:
* 12500 уникальных значений поля с идентификатором клиента Customer_ID означают то, что в наборе представлены данные 12500 клиентов.
* 8 уникальных значений поля с месяцем Month означают то, что данные представлены не за все месяцы.
* 1788 уникальных значений поля с возрастом Age выглядят странно.
* 12501 уникальных значений поля с номером социального страхования SNN при наличии данных о 12500 клиентах означают то, что у одного из клиентов представлен неправильный SNN.
* Данные содержат лишние символы, например, "_".

##### Удаление полей, не влияющих на кредитный рейтинг

In [244]:
df.drop(['ID', 'Customer_ID', 'Name', 'SSN', 'Month'], axis=1, inplace=True)

##### Удаление лишних символов из данных

In [245]:
def text_cleaning(data):
    if data is np.NaN or not isinstance(data, str):
        return data
    else:
        return str(data).strip('_ ,"')

In [246]:
df = df.map(text_cleaning)

##### Замена не имеющих смысла значений на NaN 

In [247]:
df = df.replace(['', 'nan', '!@9#%8'], np.NaN)

##### Определение типов данных

In [248]:
df.dtypes

Age                          object
Occupation                   object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour            object
Monthly_Balance              object
Credit_Score                 object
dtype: object

Выводы:
* Данные представлены строковым и числовым типом.
* Числовые поля Age, Annual_Income, Num_of_Loan, Num_of_Delayed_Payment, Changed_Credit_Limit, Outstanding_Debt, Amount_invested_monthly, Monthly_Balance представлены неверным типом.

##### Приведение типов

In [249]:
wrong_types = ['Age', 'Annual_Income', 'Num_of_Loan', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
               'Outstanding_Debt', 'Amount_invested_monthly', 'Monthly_Balance'
            ]

In [250]:
for col in wrong_types:
    df[col] = df[col].astype('float64')

С помощью функции convert_to_month() конвертируем значения поля Credit_History_Age, представленные годами и месяцами, в месяцы (например, "2 Years and 7 Months" в "19") и преобразуем в тип float64.

In [251]:
def convert_to_months(history):
    if pd.notnull(history):
        years = int(history.split(' ')[0])
        months = int(history.split(' ')[3])
        return (years * 12) + months
    return 0

In [252]:
df['Credit_History_Age'] = df.Credit_History_Age.apply(lambda x: convert_to_months(x)).astype(float)

##### Обработка отрицательных значений

In [253]:
numerical_columns = df.select_dtypes(include=['float64','int64']).columns.to_list()

for col in numerical_columns:
    df[col] = df[col].map(lambda x: np.NaN if x < 0 else x)

##### Определение значений категориальных переменных

In [267]:
categorical = df.select_dtypes(include=['object'])
categorical_columns = categorical.columns.to_list()
for col in categorical_columns:
    print(df[col].value_counts())
    print('_______________________')

Occupation
Lawyer           6575
Architect        6355
Engineer         6350
Scientist        6299
Mechanic         6291
Accountant       6271
Developer        6235
Media_Manager    6232
Teacher          6215
Entrepreneur     6174
Doctor           6087
Journalist       6085
Manager          5973
Musician         5911
Writer           5885
Name: count, dtype: int64
_______________________
Type_of_Loan
Not Specified                                                                                                                         1408
Credit-Builder Loan                                                                                                                   1280
Personal Loan                                                                                                                         1272
Debt Consolidation Loan                                                                                                               1264
Student Loan                            

#### Преобразование данных

В поле Age представлено 1788 уникальных значений, но возраст должны быть целым числом от 18 до 85 (максимальный возраст заёмщика).

In [254]:
df['Age'].value_counts()

Age
38.0      2994
28.0      2968
31.0      2955
26.0      2945
32.0      2884
          ... 
1908.0       1
4583.0       1
7549.0       1
3119.0       1
1342.0       1
Name: count, Length: 1727, dtype: int64

In [255]:
for i in range(len(df)):
    if df.loc[i, 'Age'] > 85 or df.loc[i, 'Age'] < 18 :
        df.loc[i, 'Age'] = np.NaN  

В поле Payment_of_Min_Amount значение NM является неправильным.

In [270]:
df.loc[df['Payment_of_Min_Amount'] == 'NM', 'Payment_of_Min_Amount'] = np.nan

##### Обработка пропущенных значений

In [271]:
print(f"Пропущенные значения:\n{df.isnull().sum()}")

Пропущенные значения:
Age                          8487
Occupation                   7062
Annual_Income                   0
Monthly_Inhand_Salary           0
Num_Bank_Accounts              21
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                  3876
Type_of_Loan                11408
Delay_from_due_date           591
Num_of_Delayed_Payment          0
Changed_Credit_Limit            0
Num_Credit_Inquiries            0
Credit_Mix                  20195
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age              0
Payment_of_Min_Amount       12007
Total_EMI_per_month             0
Amount_invested_monthly         0
Payment_Behaviour            7600
Monthly_Balance                 0
Credit_Score                    0
Credit_Score_Encoded            0
Occupation_Encoded              0
dtype: int64


Пропущенные значения в полях Monthly_Inhand_Salary, Num_of_Delayed_Payment, Changed_Credit_Limit, Num_Credit_Inquiries, Amount_invested_monthly и Monthly_Balance рассчитаем как среднее арифметическое.

In [272]:
null_num_cols = ['Monthly_Inhand_Salary', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Amount_invested_monthly', 'Monthly_Balance']
imputer = SimpleImputer(strategy='mean')

df[null_num_cols] = pd.DataFrame(imputer.fit_transform(df[null_num_cols]),columns=null_num_cols)


#### 4 Кодирование категориальных переменных

##### Определение категориальных переменных

In [258]:
df.describe(exclude=np.number).T

,count,unique,top,freq
Occupation,92938,15,Lawyer,6575
Type_of_Loan,88592,6260,Not Specified,1408
Credit_Mix,79805,3,Standard,36479
Payment_of_Min_Amount,100000,3,Yes,52326
Payment_Behaviour,92400,6,Low_spent_Small_value_payments,25513
Credit_Score,100000,3,Standard,53174


In [259]:
categorical = df.select_dtypes(include=['object'])
categorical_columns = categorical.columns.to_list()
categorical.head()

,Occupation,Type_of_Loan,Credit_Mix,Payment_of_Min_Amount,Payment_Behaviour,Credit_Score
0,Scientist,"Auto Loan, Credit-Builder Loan, Personal Loan,...",NaN,No,High_spent_Small_value_payments,Good
1,Scientist,"Auto Loan, Credit-Builder Loan, Personal Loan,...",Good,No,Low_spent_Large_value_payments,Good
2,Scientist,"Auto Loan, Credit-Builder Loan, Personal Loan,...",Good,No,Low_spent_Medium_value_payments,Good
3,Scientist,"Auto Loan, Credit-Builder Loan, Personal Loan,...",Good,No,Low_spent_Small_value_payments,Good
4,Scientist,"Auto Loan, Credit-Builder Loan, Personal Loan,...",Good,No,High_spent_Medium_value_payments,Good


##### Кодирование категориальных переменных

**Credit_Score_Encoded**

In [260]:
categories = ['Poor', 'Standard', 'Good']
encoder = OrdinalEncoder(categories=[categories])
df['Credit_Score_Encoded'] = encoder.fit_transform(df[['Credit_Score']])
df['Credit_Score_Encoded'].value_counts()


Credit_Score_Encoded
1.0    53174
0.0    28998
2.0    17828
Name: count, dtype: int64

**Occupation**

In [261]:
label_encoder = LabelEncoder()
df['Occupation_Encoded'] = label_encoder.fit_transform(df['Occupation'])
df['Occupation_Encoded'].value_counts()

Occupation_Encoded
15    7062
7     6575
1     6355
4     6350
12    6299
9     6291
0     6271
2     6235
10    6232
13    6215
5     6174
3     6087
6     6085
8     5973
11    5911
14    5885
Name: count, dtype: int64

**Credit_Mix_Encoded**

In [262]:
categories = ['Bad', 'Standard', 'Good']
encoder = OrdinalEncoder(categories=[categories])
df['Credit_Mix_Encoded'] = encoder.fit_transform(df[['Credit_Mix']])
df['Credit_Mix_Encoded'].value_counts()

ValueError: Found unknown categories [nan] in column 0 during fit

**Payment Behaviour**

In [ ]:
df = df.dropna(subset=['Payment_Behaviour'])

categories_payment_behaviour = [
    'Low_spent_Small_value_payments', 
    'Low_spent_Medium_value_payments', 
    'Low_spent_Large_value_payments', 
    'High_spent_Small_value_payments', 
    'High_spent_Medium_value_payments', 
    'High_spent_Large_value_payments'
]

encoder_payment_behaviour = OrdinalEncoder(categories=[categories_payment_behaviour])
df['Payment_Behaviour_Encoded'] = encoder_payment_behaviour.fit_transform(categorical[['Payment_Behaviour']])

df['Payment_Behaviour_Encoded'].value_counts()


ValueError: Found unknown categories [nan] in column 0 during fit

##### Удаление закодированных столбцов

In [ ]:
columns_to_drop = [ 'Payment_Behaviour', 'Credit_Mix', 'Occupation','Credit_Score']
df.drop(columns=columns_to_drop, inplace=True)

KeyError: "['Payment_Behaviour', 'Credit_Mix', 'Occupation', 'Credit_Score'] not found in axis"